In [1]:
from sklearn.tree import export_graphviz
import pydot

In [21]:
#Import dependencies
import pandas as pd
import sqlite3
import argparse
import json
import pprint
import requests
import sys
import urllib
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode
import matplotlib.pyplot as plt
import csv
import os

In [22]:
#Assign database file to a variable
db_file = "../Resources/Los_Angeles/crimedata.db"

#Connect to database file
conn = sqlite3.connect(db_file)

In [50]:
#Query the la crime data table and put into a pandas dataframe
crime_df = pd.read_sql_query("select * from la_crime_1718;", conn)
crime_df.head()

,DRNumber,DateReported,DateOccurred,TimeOccurred,AreaID,AreaName,ReportingDistrict,CrimeCode,CrimeCodeDescription,MOCodes,...,WeaponDescription,StatusCode,StatusDescription,CrimeCode1,CrimeCode2,CrimeCode3,CrimeCode4,Address,CrossStreet,Location
0,170209449,4/22/2017,4/21/2017,1930,2,Rampart,201,510,VEHICLE - STOLEN,,...,,IC,Invest Cont,510,,,,ALEXANDRIA,ROMAINE,"(34.0886, -118.2979)"
1,170206243,2/14/2017,2/11/2017,1700,2,Rampart,275,510,VEHICLE - STOLEN,,...,,IC,Invest Cont,510,,,,OLYMPIC,WESTLAKE,"(34.0512, -118.2787)"
2,170311599,4/25/2017,4/25/2017,745,3,Southwest,325,510,VEHICLE - STOLEN,,...,,IC,Invest Cont,510,,,,VERMONT,ADAMS,"(34.0328, -118.2915)"
3,170408264,4/7/2017,4/7/2017,1,4,Hollenbeck,421,510,VEHICLE - STOLEN,,...,,IC,Invest Cont,510,,,,ALHAMBRA,VALLEY,"(34.0676, -118.2202)"
4,170508584,4/10/2017,4/8/2017,730,5,Harbor,566,510,VEHICLE - STOLEN,,...,,IC,Invest Cont,510,,,,10TH,CENTURY,"(33.7347, -118.2842)"


In [24]:
crime_df.columns

Index(['DRNumber', 'DateReported', 'DateOccurred', 'TimeOccurred', 'AreaID',
       'AreaName', 'ReportingDistrict', 'CrimeCode', 'CrimeCodeDescription',
       'MOCodes', 'VictimAge', 'VictimSex', 'VictimDescent', 'PremiseCode',
       'PremiseDescription', 'WeaponUsedCode', 'WeaponDescription',
       'StatusCode', 'StatusDescription', 'CrimeCode1', 'CrimeCode2',
       'CrimeCode3', 'CrimeCode4', 'Address', 'CrossStreet', 'Location'],
      dtype='object')

In [51]:
# Drop unnecessary crime columns for machine learning
crime_df = crime_df.drop(['DRNumber','DateReported', 
                          'DateOccurred','TimeOccurred',
                          'AreaName','CrimeCodeDescription',
                          'MOCodes','VictimSex',
                          'WeaponUsedCode','PremiseDescription',
                          'WeaponDescription','VictimDescent',
                          'StatusCode', 'StatusDescription',
                          'Address', 'CrossStreet',
                          'CrimeCode1', 'CrimeCode2',
                          'CrimeCode3', 'CrimeCode4','Location'
                         ], axis=1)
crime_df.head()

,AreaID,ReportingDistrict,CrimeCode,VictimAge,PremiseCode
0,2,201,510,16,101
1,2,275,510,16,101
2,3,325,510,16,101
3,4,421,510,16,101
4,5,566,510,16,101


In [52]:
crime_df.count()

AreaID               174766
ReportingDistrict    174766
CrimeCode            174766
VictimAge            174766
PremiseCode          174766
dtype: int64

In [53]:
# Most victim age crimes are commited against
ages = crime_df.VictimAge.value_counts()
ages.head(10)

17    12516
16    12093
      11503
26     4411
27     4323
25     4271
28     4179
29     4131
24     4063
30     3966
Name: VictimAge, dtype: int64

In [54]:
area_ID = crime_df.AreaID.value_counts() 
area_ID.head(10)

3     26280
1     24183
6     20940
7     18542
2     18192
9     17306
5     16300
4     15293
8     15290
13      816
Name: AreaID, dtype: int64

In [55]:
premise_desc = crime_df.PremiseCode.value_counts() 
premise_desc.head()

101    38064
502    24341
501    22651
108    12778
102    11263
Name: PremiseCode, dtype: int64

In [56]:
crime_code = crime_df.CrimeCode.value_counts() 
crime_code.head(10)

624    16820
330    14673
440    13046
510    11412
310    10286
740    10077
626     8903
230     8644
210     7344
420     6942
Name: CrimeCode, dtype: int64

In [156]:
# model_small_df = crime_df.loc[crime_df['CrimeCode'].isin(['624'])]
# 440 - 0.628928407174613
model_small_df = crime_df.loc[crime_df['AreaID'].isin(['7'])]

model_small_df.head()

,AreaID,ReportingDistrict,CrimeCode,VictimAge,PremiseCode
5,7,727,510,16,101
36,7,736,510,16,123
27893,7,724,510,16,707
28019,7,774,510,16,101
28150,7,701,480,34,108


In [157]:
model_small_df.count()

AreaID               18542
ReportingDistrict    18542
CrimeCode            18542
VictimAge            18542
PremiseCode          18542
dtype: int64

In [158]:
#COnvert the datatype of each column to numeric for machine learning
model_small_df['AreaID'] = model_small_df['AreaID'].apply(pd.to_numeric, errors='coerce')
model_small_df['CrimeCode'] = model_small_df['CrimeCode'].apply(pd.to_numeric, errors='coerce')
model_small_df['ReportingDistrict'] = model_small_df['ReportingDistrict'].apply(pd.to_numeric, errors='coerce')
model_small_df['VictimAge'] = model_small_df['VictimAge'].apply(pd.to_numeric, errors='coerce')
model_small_df['PremiseCode'] = model_small_df['PremiseCode'].apply(pd.to_numeric, errors='coerce')

model_small_df = model_small_df.fillna(0)
model_small_df.reset_index()
model_small_df.head()

C:\Users\sherr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\sherr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sherr\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,AreaID,ReportingDistrict,CrimeCode,VictimAge,PremiseCode
5,7,727,510,16.0,101
36,7,736,510,16.0,123
27893,7,724,510,16.0,707
28019,7,774,510,16.0,101
28150,7,701,480,34.0,108


In [159]:
X = model_small_df.drop("CrimeCode", axis=1)
y = model_small_df["CrimeCode"]
feature_names = model_small_df.columns
print(X.shape, y.shape)

(18542, 4) (18542,)


In [160]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=20)
rf = rf.fit(X, y)
rf.score(X, y)

0.7205263725595944

In [136]:
importances = rf.feature_importances_
importances

array([0.        , 0.29236349, 0.44085382, 0.26678269])

In [126]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True) 

[(0.7108896855141416, 'CrimeCode'),
 (0.15545536105455987, 'VictimAge'),
 (0.13365495343129868, 'AreaID'),
 (0.0, 'ReportingDistrict')]

In [127]:
 from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
 # Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

In [ ]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train, y_train)

In [ ]:
# List the best parameters for this dataset
print(grid.best_params_)

In [ ]:
 # List the best score
print(grid.best_score_)

In [ ]:
 # Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [161]:
from sklearn.tree import export_graphviz
import pydot

#Extract one of the trees
tree = rf.estimators_[3]

#Save the tree as a png
export_graphviz(tree, out_file='crime_tree.dot', feature_names = feature_names, rounded=True, precision=1)

(graph, ) = pydot.graph_from_dot_file('crime_tree.dot')

graph.write_png('crime_tree.png')

ModuleNotFoundError: No module named 'pydot'